## Report Generation
This notebook serves as continuation of earlier notebooks (i.e. [Article Classifier](./News%20Classifier/FYP%20Article%20Classification%20Codes.ipynb) and [3D Plot Visualization](./3D%20Plotting%20and%20Visualization/3D%20Plot%20Visualization.ipynb))  

### Import Necessary Libraries
Spacy was used for its Named Entity Recognition (NER) text processing in reporting the content summary.  
Requests was used in making HTTP requests to Nominatim in retrieving longitude and latitude data for the countries identified.  
Ipyleaflet was used in generating the interactive map as part of the report generated.

In [17]:
import spacy
from spacy import displacy

# !pip3 install requests
import requests
# !pip install ipyleaflet
# !jupyter nbextension enable --py --sys-prefix ipyleaflet
from ipyleaflet import Map, Marker, AwesomeIcon, CircleMarker, LayerGroup, LegendControl

For stakeholders who are unsure of what the labels (e.g. GPE, LOC) mean, they can simply run <b>spacy.explain(\<label\>)</b> to get the official definition by spaCy. For instance (see below), <b>spacy.explain("GPE")</b> returns "Countries, cities, states" suggesting that the entities highlighted by spaCy and labeled as 'GPE' refer to either countries, cities or states.

In [25]:
spacy.explain("GPE")

'Countries, cities, states'

#### Function to display the interactive map:
1. To obtain all of the countries mentioned in the content summary
2. For all of these countries, get their longtitude and latitude data
3. Instantiate the map object, with markers corresponding to each of the countries

In [19]:
def displayMap(doc): # takes in the NER processed text!
    # get all countries mentioned
    countries = {}
    for word in doc.ents:
        if word.label_ == "GPE" and word.text not in countries:
            countries[word.text] = []
#     print(countries)
    
    # get all longitude and latitude of countries mentioned
    for country,center in countries.items():
        url = f"http://nominatim.openstreetmap.org/search?country={country}&format=json&polygon=0"
        response = requests.get(url).json()[0]
        countries[country] = [response['lat'], response['lon']]
#     print(countries)

    # finding the center of all countries - for display sake!
    sumLat = 0
    sumLon = 0
    for geocode in countries.values():
        sumLat += float(geocode[0])    
        sumLon += float(geocode[1])
    center = [sumLat/len(countries), sumLon/len(countries)]
    
    # instantiating an instance of the map
    currMap=Map(center=center, zoom=0)
    icon1 = AwesomeIcon(
        name='exclamation',
        marker_color='red',
        icon_color='white',
        spin=False
    )
    
    layers = [];    
    for country, data in countries.items():
        marker = Marker(icon=icon1, location=data, title=country)
        layers.append(marker)
    
    layer_group = LayerGroup(layers=tuple(layers))
    currMap.add_layer(layer_group)
    currMap

    return

#### Function to generate report: 
Ideally, this function takes in one row of the resulting dataframe and retrive the relevant information, to finally generate a report, which includes the visualziation of the interactive map mentioned above. 

Relevant information includes: Date, Title, Source(URL), Content Summary, Trigger Event, Stages of Supply Chain that was affected, Risk Level

In [126]:
def report_writer(row):

    #assign values into callable instances
    date = row['date']
    title = row["news title"]
    url = row["news source(url)"]
    
    nlp = spacy.load('en_core_web_sm')
    current_text = row['content summary']
    doc = nlp(current_text)
    
    print (f"***************************START OF REPORT********************************")
    print ("\n")

    print (f"This is an automatically generated report for:")
    print (f"\n{colored('News Title: ', attrs=['bold']) + title}'\n{colored('Dated: ', attrs=['bold']) + date}\n{colored('News Source (URL): ', attrs=['bold']) + url}.")
    print ("\n")
    
    print(f"{colored('Trigger Event Identified: ', attrs=['bold']) + output[1].iloc[0].name} with a probability of {round(output[1].iloc[0].Probability, 2)}%")    
    
    print(f"{colored('Stage(s) of supply chain likely to be affected: ', attrs=['bold']) + output[2]}")
    
    # color to be returned from the cube - yellow, red, green
    risk = colored('High', 'red', attrs=['reverse','bold'])
    print(f"{colored('Risk Level Identified: ', attrs=['bold']) + risk}")
    print("\n")
    
    print(f"{colored('Content Summary with following related entities highlighted in this article that should be reviewed:', attrs=['bold'])}")
    displacy.render(doc, style='ent', jupyter=True)
    print ("\n")
    
    displayMap(doc)

    print (f"***************************END OF REPORT********************************")

In [127]:
report_writer(row)

***************************START OF REPORT********************************


This is an automatically generated report for:

News Title: Ukraine tensions can choke supply of energy, food for S'pore and Asia'
Dated: undated
News Source (URL): https://www.straitstimes.com/business/companies-markets/ukraine-tensions-can-choke-supply-of-energy-food-for-spore-and-asia.


Trigger Event Identified: pandemic with a probability of 45.23%
Stage(s) of supply chain likely to be affected: N/A
Risk Level Identified: High


Content Summary with following related entities highlighted in this article that should be reviewed:




***************************END OF REPORT********************************
